In [31]:
import json
import pandas as pd
import re
import string
import numpy as np
from sklearn.pipeline import Pipeline

In [32]:
df = pd.read_csv("../data/raw/spreadsheet.csv")
df

,N,NRO_REGISTRO,FECHA_RESOLUCION,FIRMA,MATERIA,ART_INFRINGIDO,CODIGO_O_LEY,CONDUCTA,CONDUCTA_DESCRIPCION,VIOLENCIA_DE_GENERO,...,N_REGISTRO_Y_TOMO_CAMARA_2,LINK_CAMARA_2,DECISION_DE_FONDO_TSJ,N_REGISTRO_Y_TOMO_TSJ,LINK_TSJ,RECURSO_EXTRAORDINARIO_Y_RECURRENTE,DECISION_CSJN,N_REGISTRO_Y_TOMO_CSJN,LINK_CSJN,ID
0,1.0,NaN,1_8_2016,Pablo_Casas,penal,1,ley_14346,proteccion_animal,simples,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0B9wNhp3GjjazTVhZM2dCRU1IN2M
1,2.0,NaN,1_8_2016,Pablo_Casas,penal,149bis,codigo_penal_de_la_nacion,amenazas,simples,si,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0B9wNhp3GjjazTjRHVHl2YzBtUEU
2,3.0,NaN,2_8_2016,Pablo_Casas,contravencional,52,codigo_contravencional,hostigamiento,agravado_violencia_de_genero,si,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0B9wNhp3GjjazY3VCak5kdTBvWWc
3,4.0,NaN,2_8_2016,Pablo_Casas,contravencional,73,codigo_contravencional,incumplir_clausura,NaN,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0B9wNhp3Gjjazc2V4eWVaeWVRQUE
4,5.0,NaN,2_8_2016,Pablo_Casas,penal,149bis,codigo_penal_de_la_nacion,amenazas,simples,si,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0B9wNhp3GjjazZFdmVXlWbFdYSHc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3400,2783.0,2890,7_08_20,Pablo_Casas,penal,189bis,codigo_penal_de_la_nacion,tenencia_de_arma,de_fuego_uso_civil,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1wVeBMKcdaSc4w_hd8WYH7Yy6IVupN7av/view?usp=sha...
3401,2784.0,2891,8_08_20,Pablo_Casas,penal,149bis,codigo_penal_de_la_nacion,amenazas,simples,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1Zht2hB79c-c5dVS-GoSwl95RpduzyMX5/view?usp=sha...
3402,NaN,2891,8_08_20,Pablo_Casas,penal,183,codigo_penal_de_la_nacion,daños,NaN,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1Zht2hB79c-c5dVS-GoSwl95RpduzyMX5/view?usp=sha...
3403,NaN,2891,8_08_20,Pablo_Casas,penal,186_inc1,codigo_penal_de_la_nacion,incendio_explosion_inundacion_con_peligro_para...,NaN,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1Zht2hB79c-c5dVS-GoSwl95RpduzyMX5/view?usp=sha...


In [33]:
def get_id(link):
    """ Function to extract the file ID from its URL. There are two main
    Args:
        link (str): Google Drive URL for the file

    Returns:
        String with the file's ID

    """
    file_id = np.nan

    try:
        if "/file/" in link:
            file_id = link.split("/file/d/")[1]
        elif "id=" in link:
            file_id = link.split("id=")[1]
        elif "/document/" in link:
            file_id = link.split("/document/d/")[1]

    except TypeError:
        return np.nan

    # remove additional characters from the link
    if "/" in str(file_id):
        file_id = file_id.split("/")[0]

    return file_id

In [34]:
df['id'] = [get_id(link) for link in df['LINK']]
df

,N,NRO_REGISTRO,FECHA_RESOLUCION,FIRMA,MATERIA,ART_INFRINGIDO,CODIGO_O_LEY,CONDUCTA,CONDUCTA_DESCRIPCION,VIOLENCIA_DE_GENERO,...,LINK_CAMARA_2,DECISION_DE_FONDO_TSJ,N_REGISTRO_Y_TOMO_TSJ,LINK_TSJ,RECURSO_EXTRAORDINARIO_Y_RECURRENTE,DECISION_CSJN,N_REGISTRO_Y_TOMO_CSJN,LINK_CSJN,ID,id
0,1.0,NaN,1_8_2016,Pablo_Casas,penal,1,ley_14346,proteccion_animal,simples,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0B9wNhp3GjjazTVhZM2dCRU1IN2M,0B9wNhp3GjjazTVhZM2dCRU1IN2M
1,2.0,NaN,1_8_2016,Pablo_Casas,penal,149bis,codigo_penal_de_la_nacion,amenazas,simples,si,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0B9wNhp3GjjazTjRHVHl2YzBtUEU,0B9wNhp3GjjazTjRHVHl2YzBtUEU
2,3.0,NaN,2_8_2016,Pablo_Casas,contravencional,52,codigo_contravencional,hostigamiento,agravado_violencia_de_genero,si,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0B9wNhp3GjjazY3VCak5kdTBvWWc,0B9wNhp3GjjazY3VCak5kdTBvWWc
3,4.0,NaN,2_8_2016,Pablo_Casas,contravencional,73,codigo_contravencional,incumplir_clausura,NaN,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0B9wNhp3Gjjazc2V4eWVaeWVRQUE,0B9wNhp3Gjjazc2V4eWVaeWVRQUE
4,5.0,NaN,2_8_2016,Pablo_Casas,penal,149bis,codigo_penal_de_la_nacion,amenazas,simples,si,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0B9wNhp3GjjazZFdmVXlWbFdYSHc,0B9wNhp3GjjazZFdmVXlWbFdYSHc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3400,2783.0,2890,7_08_20,Pablo_Casas,penal,189bis,codigo_penal_de_la_nacion,tenencia_de_arma,de_fuego_uso_civil,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1wVeBMKcdaSc4w_hd8WYH7Yy6IVupN7av/view?usp=sha...,1wVeBMKcdaSc4w_hd8WYH7Yy6IVupN7av
3401,2784.0,2891,8_08_20,Pablo_Casas,penal,149bis,codigo_penal_de_la_nacion,amenazas,simples,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1Zht2hB79c-c5dVS-GoSwl95RpduzyMX5/view?usp=sha...,1Zht2hB79c-c5dVS-GoSwl95RpduzyMX5
3402,NaN,2891,8_08_20,Pablo_Casas,penal,183,codigo_penal_de_la_nacion,daños,NaN,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1Zht2hB79c-c5dVS-GoSwl95RpduzyMX5/view?usp=sha...,1Zht2hB79c-c5dVS-GoSwl95RpduzyMX5
3403,NaN,2891,8_08_20,Pablo_Casas,penal,186_inc1,codigo_penal_de_la_nacion,incendio_explosion_inundacion_con_peligro_para...,NaN,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1Zht2hB79c-c5dVS-GoSwl95RpduzyMX5/view?usp=sha...,1Zht2hB79c-c5dVS-GoSwl95RpduzyMX5


In [35]:
df.drop(['ART_INFRINGIDO', 'CODIGO_O_LEY', 'CONDUCTA', 'VIOLENCIA_DE_GENERO', 'FRASES_AGRESION'], axis=1, inplace=True)

In [36]:
df.V_AMB.unique()

array(['no_corresponde', 'no', 'si', 's/d'], dtype=object)

In [37]:
df = df[df['V_ECON'] != 'no_corresponde']
df

,N,NRO_REGISTRO,FECHA_RESOLUCION,FIRMA,MATERIA,CONDUCTA_DESCRIPCION,V_FISICA,V_PSIC,V_ECON,V_SEX,...,LINK_CAMARA_2,DECISION_DE_FONDO_TSJ,N_REGISTRO_Y_TOMO_TSJ,LINK_TSJ,RECURSO_EXTRAORDINARIO_Y_RECURRENTE,DECISION_CSJN,N_REGISTRO_Y_TOMO_CSJN,LINK_CSJN,ID,id
1,2.0,NaN,1_8_2016,Pablo_Casas,penal,simples,no,si,no,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0B9wNhp3GjjazTjRHVHl2YzBtUEU,0B9wNhp3GjjazTjRHVHl2YzBtUEU
2,3.0,NaN,2_8_2016,Pablo_Casas,contravencional,agravado_violencia_de_genero,si,si,si,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0B9wNhp3GjjazY3VCak5kdTBvWWc,0B9wNhp3GjjazY3VCak5kdTBvWWc
4,5.0,NaN,2_8_2016,Pablo_Casas,penal,simples,no,si,si,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0B9wNhp3GjjazZFdmVXlWbFdYSHc,0B9wNhp3GjjazZFdmVXlWbFdYSHc
9,10.0,NaN,3_8_2016,Pablo_Casas,penal,simples,si,si,no,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0B9wNhp3GjjazUG5XOWNmOVNob1U,0B9wNhp3GjjazUG5XOWNmOVNob1U
20,21.0,NaN,18_8_2016,Pablo_Casas,penal,simples,no,no,si,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0B9wNhp3GjjazR3d3V1U1LUYtS3c,0B9wNhp3GjjazR3d3V1U1LUYtS3c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3395,NaN,2886,7_08_20,Pablo_Casas,contravencional,agravado_violencia_de_genero,no,si,no,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1-1X9aRjZduRXTB86XexCTA48pQbaSM6z/view?usp=sha...,1-1X9aRjZduRXTB86XexCTA48pQbaSM6z
3397,2781.0,2888,7_08_20,Pablo_Casas,contravencional,NaN,si,si,si,si,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1rLD8fzC_O-U2yWgVmuTKi9nohGKzdl8V/view?usp=sha...,1rLD8fzC_O-U2yWgVmuTKi9nohGKzdl8V
3398,2782.0,2889,7_08_20,Pablo_Casas,penal,simples,no,si,no,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1tvUBnvDjjL2ln-vManH5ImVXxSFjXx27/view?usp=sha...,1tvUBnvDjjL2ln-vManH5ImVXxSFjXx27
3399,NaN,2889,7_08_20,Pablo_Casas,penal,simples,no,si,no,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1tvUBnvDjjL2ln-vManH5ImVXxSFjXx27/view?usp=sha...,1tvUBnvDjjL2ln-vManH5ImVXxSFjXx27


In [38]:
df.V_PSIC.unique()

array(['si', 'no', 's/d'], dtype=object)

In [39]:
df.replace('s/d', np.nan, inplace=True)
df.replace('si', 1, inplace=True)
df.replace('no', 0 , inplace=True)

/home/yabra/.local/lib/python3.8/site-packages/pandas/core/frame.py:4373: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [40]:
df.V_PSIC.unique()

array([ 1.,  0., nan])

In [41]:
target_vars = ["V_FISICA", "V_PSIC", "V_ECON", "V_SEX", "V_SOC", "V_AMB", "V_SIMB"]

In [42]:
from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer(missing_values=np.nan, strategy='median')

In [43]:
df[target_vars] = my_imputer.fit_transform(df[target_vars])

<ipython-input-43-9fffff768ee7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[target_vars] = my_imputer.fit_transform(df[target_vars])
/home/yabra/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


In [46]:
df.V_FISICA.unique()

array([0., 1.])

In [47]:
df

,N,NRO_REGISTRO,FECHA_RESOLUCION,FIRMA,MATERIA,CONDUCTA_DESCRIPCION,V_FISICA,V_PSIC,V_ECON,V_SEX,...,LINK_CAMARA_2,DECISION_DE_FONDO_TSJ,N_REGISTRO_Y_TOMO_TSJ,LINK_TSJ,RECURSO_EXTRAORDINARIO_Y_RECURRENTE,DECISION_CSJN,N_REGISTRO_Y_TOMO_CSJN,LINK_CSJN,ID,id
1,2.0,NaN,1_8_2016,Pablo_Casas,penal,simples,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0B9wNhp3GjjazTjRHVHl2YzBtUEU,0B9wNhp3GjjazTjRHVHl2YzBtUEU
2,3.0,NaN,2_8_2016,Pablo_Casas,contravencional,agravado_violencia_de_genero,1.0,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0B9wNhp3GjjazY3VCak5kdTBvWWc,0B9wNhp3GjjazY3VCak5kdTBvWWc
4,5.0,NaN,2_8_2016,Pablo_Casas,penal,simples,0.0,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0B9wNhp3GjjazZFdmVXlWbFdYSHc,0B9wNhp3GjjazZFdmVXlWbFdYSHc
9,10.0,NaN,3_8_2016,Pablo_Casas,penal,simples,1.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0B9wNhp3GjjazUG5XOWNmOVNob1U,0B9wNhp3GjjazUG5XOWNmOVNob1U
20,21.0,NaN,18_8_2016,Pablo_Casas,penal,simples,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0B9wNhp3GjjazR3d3V1U1LUYtS3c,0B9wNhp3GjjazR3d3V1U1LUYtS3c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3395,NaN,2886,7_08_20,Pablo_Casas,contravencional,agravado_violencia_de_genero,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1-1X9aRjZduRXTB86XexCTA48pQbaSM6z/view?usp=sha...,1-1X9aRjZduRXTB86XexCTA48pQbaSM6z
3397,2781.0,2888,7_08_20,Pablo_Casas,contravencional,NaN,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1rLD8fzC_O-U2yWgVmuTKi9nohGKzdl8V/view?usp=sha...,1rLD8fzC_O-U2yWgVmuTKi9nohGKzdl8V
3398,2782.0,2889,7_08_20,Pablo_Casas,penal,simples,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1tvUBnvDjjL2ln-vManH5ImVXxSFjXx27/view?usp=sha...,1tvUBnvDjjL2ln-vManH5ImVXxSFjXx27
3399,NaN,2889,7_08_20,Pablo_Casas,penal,simples,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1tvUBnvDjjL2ln-vManH5ImVXxSFjXx27/view?usp=sha...,1tvUBnvDjjL2ln-vManH5ImVXxSFjXx27
